<a href="https://colab.research.google.com/github/Subhash11520/PRODIGY_GA_05/blob/main/Neural_Style__Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Neural Style Transfer Task 5:
from google.colab import files
import IPython.display as display
from IPython.display import clear_output
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import ipywidgets as widgets
import os

def load_img(path_to_img, max_dim):
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

def run_style_transfer(content_path, style_path, max_dim):
    content_image = load_img(content_path, max_dim)
    style_image = load_img(style_path, max_dim)
    stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
    return content_image, style_image, stylized_image

def show_images(content, style, stylized):
    content_img = tensor_to_image(content)
    style_img = tensor_to_image(style)
    stylized_img = tensor_to_image(stylized)

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    for ax, img, title in zip(axes, [content_img, style_img, stylized_img],
                              ["Content Image", "Style Image", "Stylized Image"]):
        ax.imshow(img)
        ax.set_title(title)
        ax.axis('off')
    plt.show()

content_uploader = widgets.FileUpload(accept='image/*', multiple=False)
style_uploader = widgets.FileUpload(accept='image/*', multiple=False)
resolution_slider = widgets.IntSlider(value=512, min=256, max=1024, step=64, description='Max Res:')
download_checkbox = widgets.Checkbox(value=True, description='Enable Download')
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Progress:', bar_style='info')
output_box = widgets.Output()

def on_button_clicked(b):
    with output_box:
        clear_output()
        print("Running style transfer...")
        progress_bar.value = 0.1

        content_path = None
        style_path = None

        for name, file_info in content_uploader.value.items():
            content_path = "/content/content_img.jpg"
            with open(content_path, 'wb') as f:
                f.write(file_info['content'])

        for name, file_info in style_uploader.value.items():
            style_path = "/content/style_img.jpg"
            with open(style_path, 'wb') as f:
                f.write(file_info['content'])

        if not content_path or not style_path:
            print("Please upload both content and style images.")
            return

        progress_bar.value = 0.4
        content, style, result = run_style_transfer(content_path, style_path, resolution_slider.value)
        progress_bar.value = 0.8

        show_images(content, style, result)
        progress_bar.value = 1.0

        if download_checkbox.value:
            result_img = tensor_to_image(result)
            stylized_path = "/content/stylized_output.png"
            result_img.save(stylized_path)
            print(f"\nStylized image saved as: {stylized_path}")
            files.download(stylized_path)

apply_button = widgets.Button(description='Apply Style Transfer', button_style='success')
apply_button.on_click(on_button_clicked)

ui = widgets.VBox([
    widgets.HTML("<h2>Neural Style Transfer UI</h2>"),
    widgets.HBox([widgets.Label("Upload Content:"), content_uploader]),
    widgets.HBox([widgets.Label("Upload Style:  "), style_uploader]),
    resolution_slider,
    download_checkbox,
    apply_button,
    progress_bar,
    output_box
])

display.display(ui)
